In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)

N_TRAJECTORIES = 5000
TRAJECTORY_LENGTH = 257
VALIDATION_SPLIT = 0.2  # fraction of the training data used for validation

In [5]:
#1285000 / 257 = 5000 trajectories
data = pd.read_csv("X_train.csv")
n_lines = data.shape[0]
assert n_lines / TRAJECTORY_LENGTH == N_TRAJECTORIES

### Train/Validation Split

In [6]:
#array([0, 1, ..., 4999]) with IDs of all trajectories
all_trajectories = np.arange(N_TRAJECTORIES)
#number of trajectories (e.g., 1000) used for validation
n_trajectories_val = int(N_TRAJECTORIES * 0.2)
#indices (from 0 to 4999) that identify the validation trajectories
trajectories_val = np.random.choice(N_TRAJECTORIES, n_trajectories_val, replace=False)
#the train trajectories IDs are those remaining from removing the validation trajectories from all 5000
trajectories_train = np.setdiff1d(all_trajectories, trajectories_val)
n_trajectories_train = N_TRAJECTORIES - n_trajectories_val

#now we find the indices of the rows corresponding to each set (train and validation) of trajectories
validation_indices = trajectories_val.repeat(TRAJECTORY_LENGTH - 1) * (TRAJECTORY_LENGTH - 1)
validation_indices += np.tile(np.arange(TRAJECTORY_LENGTH - 1), n_trajectories_val)
train_indices = trajectories_train.repeat(TRAJECTORY_LENGTH - 1) * (TRAJECTORY_LENGTH - 1)
train_indices += np.tile(np.arange(TRAJECTORY_LENGTH - 1), n_trajectories_train)

In [7]:
X = data.iloc[all_trajectories * TRAJECTORY_LENGTH][["x_1", "y_1", "x_2", "y_2", "x_3", "y_3"]]
X = X.to_numpy().repeat(TRAJECTORY_LENGTH - 1, axis=0)
X = np.concatenate((X, data[data.index % TRAJECTORY_LENGTH != 0][["t"]].to_numpy()), axis=1)

y = data[data.index % TRAJECTORY_LENGTH != 0][["x_1", "y_1", "x_2", "y_2", "x_3", "y_3"]].to_numpy()

X_train, y_train = X[train_indices], y[train_indices]
X_train_remove = np.where(~X_train[:, :-1].any(axis=1))[0]
y_train_remove = np.where(~y_train.any(axis=1))[0]
train_remove = np.concatenate((X_train_remove, y_train_remove))
X_train, y_train = np.delete(X_train, train_remove, axis=0), np.delete(y_train, train_remove, axis=0)
X_val, y_val = X[validation_indices], y[validation_indices]
X_val_remove = np.where(~X_val[:, :-1].any(axis=1))[0]
y_val_remove = np.where(~y_val.any(axis=1))[0]
val_remove = np.concatenate((X_val_remove, y_val_remove))
X_val, y_val = np.delete(X_val, val_remove, axis=0), np.delete(y_val, val_remove, axis=0)

### Baseline

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [12]:
pipeline_baseline = Pipeline([("scaler", StandardScaler()), ("lr", LinearRegression())])
pipeline_baseline.fit(X_train, y_train)
y_pred = pipeline_baseline.predict(X_val)
rms = root_mean_squared_error(y_pred, y_val)
rms

1.315613931935684

In [14]:
#test set
def prepare_submission(pipeline, out_filename):
    data_test = pd.read_csv("X_test.csv")
    columns = data_test.columns.tolist()
    id_column = data_test[columns[0]]
    X_test_columns = columns[2:] + [columns[1]]
    X_test = data_test[X_test_columns]

    y_test = pipeline.predict(X_test)
    y_test_df = pd.DataFrame(y_test, columns=["x_1", "y_1", "x_2", "y_2", "x_3", "y_3"])
    y_test_df.insert(0, "Id", id_column)
    y_test_df.to_csv(out_filename, index=False)

In [ ]:
# Assuming y_val is your data with 6 columns (replace this with your actual data)
df = pd.DataFrame(y_val, columns=["x_1", "y_1", "x_2", "y_2", "x_3", "y_3"])

# Create a pairplot (taking a sample of 200 rows for efficiency)
pairplot = sns.pairplot(df.sample(200), kind="hist")

# Save the pairplot as an image
pairplot.savefig('pairplot_image.png', format='png', dpi=300)  # Save as PNG with high resolution

# Show the plot (optional, if you want to display it as well)
plt.show()

In [ ]:
plot_y_yhat(y_val,y_pred, plot_title = "plot")

In [ ]:
residuals_dict = {
    'x_1': residuals[:, 0],
    'y_1': residuals[:, 1],
    'x_2': residuals[:, 2],
    'y_2': residuals[:, 3],
    'x_3': residuals[:, 4],
    'y_3': residuals[:, 5]
}

# Create a density plot for each residual (x_1, y_1, x_2, y_2, x_3, y_3)
plt.figure(figsize=(12, 8))

for i, (label, residual) in enumerate(residuals_dict.items()):
    plt.subplot(3, 2, i+1)
    sns.kdeplot(residual, fill=True, color='blue')
    plt.title(f"Residual Density Plot for {label}")
    plt.xlabel("Residuals")
    plt.ylabel("Density")

plt.tight_layout()
plt.show()

In [ ]:
prepare_submission(pipeline_baseline, "baseline-model.csv")

### Polynomial Regression

In [17]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV

In [19]:
def validate_poly_regression(X_train, y_train, X_val, y_val, regressor=None, degrees=range(1, 15), max_features=None):
    best_rms = np.inf
    best_model = None
    best_degree = 1
    for d in degrees:
        pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("poly", PolynomialFeatures(degree=d)),
            ("regressor", regressor)
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)
        rms = root_mean_squared_error(y_pred, y_val)
        print(f"Degree {d}:\t{rms}")
        if rms < best_rms:
            best_rms = rms
            best_model = pipeline
            best_degree = d
    return best_model, best_rms, best_degree

# fraction of the training data used for validating polynomial regression
TRAIN_SUBSET = 0.05

#10 random train subsets and rmse for each polynomial degree
n_examples_poly = int(X_train.shape[0] * TRAIN_SUBSET)
degrees = []
for _ in range(10):
    examples_poly = np.random.choice(X_train.shape[0], n_examples_poly, replace=False)
    X_poly, y_poly = X_train[examples_poly], y_train[examples_poly]
    _, _, degree = validate_poly_regression(X_poly, y_poly, X_val, y_val, regressor=LinearRegression(), degrees=range(1, 6))
    degrees.append(degree)

# - Escolhe o melhor grau de PolynomialFeatures, e testa numa célula abaixo um pipeline com vários valores de RidgeCV.

Degree 1:	1.315661963520277
Degree 2:	1.2797359713150513
Degree 3:	1.2396127925990426
Degree 4:	1.1973448374899642
Degree 5:	1.1600737247120254
Degree 1:	1.315745769609998
Degree 2:	1.2803953969413397
Degree 3:	1.2517200751204756
Degree 4:	1.1976992916077231
Degree 5:	1.1588309031281439
Degree 1:	1.3156630991501936
Degree 2:	1.2795984328502552
Degree 3:	1.2394853888381165
Degree 4:	1.1978607545310953
Degree 5:	1.1607230508596411
Degree 1:	1.3156513201476983
Degree 2:	1.2803656865978421
Degree 3:	1.2405431111314071
Degree 4:	1.1992263307651532
Degree 5:	1.1609827529538441
Degree 1:	1.3158077518290952
Degree 2:	1.2800804150320353
Degree 3:	1.240582932072795
Degree 4:	1.1984752733993005
Degree 5:	1.1597857447840225
Degree 1:	1.315813644999093
Degree 2:	1.2806978297100966
Degree 3:	1.2416043785548516
Degree 4:	1.2015242106474917
Degree 5:	1.1640306387463453
Degree 1:	1.3158248239855592
Degree 2:	1.2808579122592583
Degree 3:	1.241085720453982
Degree 4:	1.1990294740414533
Degree 5:	1.1752075

In [ ]:
plot_y_yhat(y_val,y_pred, plot_title = "plot")

In [ ]:
residuals_dict = {
    'x_1': residuals[:, 0],
    'y_1': residuals[:, 1],
    'x_2': residuals[:, 2],
    'y_2': residuals[:, 3],
    'x_3': residuals[:, 4],
    'y_3': residuals[:, 5]
}

# Create a density plot for each residual (x_1, y_1, x_2, y_2, x_3, y_3)
plt.figure(figsize=(12, 8))

for i, (label, residual) in enumerate(residuals_dict.items()):
    plt.subplot(3, 2, i+1)
    sns.kdeplot(residual, fill=True, color='blue')
    plt.title(f"Residual Density Plot for {label}")
    plt.xlabel("Residuals")
    plt.ylabel("Density")

plt.tight_layout()
plt.show()

In [ ]:
from math import comb

In [27]:
#number of features according to polynomial degree
def num_polynomial_features(n_features, degree):
    num_features = 0
    for d in range(degree + 1):
        num_features += comb(n_features + d - 1, d)
    return num_features

n_features = 7 #number of columns in X_train
degree = 3
print(num_polynomial_features(n_features, degree))  #120 features
degree = 4
print(num_polynomial_features(n_features, degree))  #330 features
degree = 5
print(num_polynomial_features(n_features, degree))  #792 features

120
330
792


In [33]:
#RidgeCV: select this from the experiments above
#done for 3/5 degree
poly_degree = 3
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("poly", PolynomialFeatures(degree=poly_degree)),
    ("regressor", RidgeCV(alphas=[0.1, 1.0, 10.0], scoring="neg_root_mean_squared_error"))
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
rms = root_mean_squared_error(y_pred, y_val)
print(rms)
#Ridge doesn't seem to make difference, could be explained due to a low degree polynomial

1.2398491225107946


In [34]:
ridge_cv = pipeline.named_steps['regressor']
print(f"best alpha:\t{ridge_cv.alpha_}")
print(f"best score:\t{ridge_cv.best_score_}")

best alpha:	10.0
best score:	-1.3421121858473832


In [ ]:
#polynomial regression chosen: 3 and 5
#7 too much for the computer to handle
pipeline_poly = Pipeline([
    ("scaler", StandardScaler()),
    ("poly", PolynomialFeatures(degree=5)),
    ("regressor", LinearRegression())
])
pipeline_poly.fit(X_train, y_train)
prepare_submission(pipeline_poly, "polynomial_submission.csv")
y_pred = pipeline_poly.predict(X_val)
rms = root_mean_squared_error(y_pred, y_val)
print(rms)

In [ ]:
plot_y_yhat(y_val,y_pred, plot_title = "plot")

In [ ]:
residuals_dict = {
    'x_1': residuals[:, 0],
    'y_1': residuals[:, 1],
    'x_2': residuals[:, 2],
    'y_2': residuals[:, 3],
    'x_3': residuals[:, 4],
    'y_3': residuals[:, 5]
}

# Create a density plot for each residual (x_1, y_1, x_2, y_2, x_3, y_3)
plt.figure(figsize=(12, 8))

for i, (label, residual) in enumerate(residuals_dict.items()):
    plt.subplot(3, 2, i+1)
    sns.kdeplot(residual, fill=True, color='blue')
    plt.title(f"Residual Density Plot for {label}")
    plt.xlabel("Residuals")
    plt.ylabel("Density")

plt.tight_layout()
plt.show()

## Feature Engineering


In [19]:
from sklearn.preprocessing import FunctionTransformer

In [27]:
# create new features (distances between each of the three bodies)
def add_features(X):
    # pairwise distances
    d_1_2 = np.linalg.norm(X[:, 0:2] - X[:, 2:4], axis=1).reshape(-1, 1)
    d_1_3 = np.linalg.norm(X[:, 0:2] - X[:, 4:6], axis=1).reshape(-1, 1)
    d_2_3 = np.linalg.norm(X[:, 2:4] - X[:, 4:6], axis=1).reshape(-1, 1)
    # pairwise angles
    a_1_2 = np.arctan((X[:, 3] - X[:, 1]) / (X[:, 2] - X[:, 0])).reshape(-1, 1)
    a_1_3 = np.arctan((X[:, 5] - X[:, 1]) / (X[:, 4] - X[:, 0])).reshape(-1, 1)
    a_2_3 = np.arctan((X[:, 5] - X[:, 3]) / (X[:, 4] - X[:, 0])).reshape(-1, 1)
    # center of mass
    cm = X[:, 0:2] + X[:, 2:4] + X[:, 4:6]

    return np.hstack([X, d_1_2, d_1_3, d_2_3, a_1_2, a_1_3, a_2_3, cm])

def remove_features(X):
    return np.delete(X, [0, 2, 4], axis=1)

pipeline_eng = Pipeline([
    ("scaler", StandardScaler()),
    ("distance_features", FunctionTransformer(add_features)),
    ("poly", PolynomialFeatures(degree=2)),
    ("regressor", LinearRegression())
])

#pipeline = Pipeline([
#    ("scaler", StandardScaler()),
    #("distance_features", FunctionTransformer(add_features)),
#    ("remove_features", FunctionTransformer(remove_features)),
#    ("regressor", LinearRegression())
#])

pipeline_eng.fit(X_train, y_train)
y_pred = pipeline_eng.predict(X_val)
rms = root_mean_squared_error(y_pred, y_val)
print(rms)
# no feats -> 1.316
# only distances -> 1.301
# distances + angles -> 1.290
# distances + angles + cm -> 1.290
# P2 + only distances -> 1.252
# P2 + distances + angles -> 1.268
# P2 + distances + angles + cm -> 1.205 (13.8s)
# P2 + distances + cm -> 1.257
#
#
# FINAL: maybe do P3/4 + distances + angles + cm?

1.1996066255538815


In [ ]:
plot_y_yhat(y_val,y_pred, plot_title = "plot")

In [ ]:
residuals_dict = {
    'x_1': residuals[:, 0],
    'y_1': residuals[:, 1],
    'x_2': residuals[:, 2],
    'y_2': residuals[:, 3],
    'x_3': residuals[:, 4],
    'y_3': residuals[:, 5]
}

# Create a density plot for each residual (x_1, y_1, x_2, y_2, x_3, y_3)
plt.figure(figsize=(12, 8))

for i, (label, residual) in enumerate(residuals_dict.items()):
    plt.subplot(3, 2, i+1)
    sns.kdeplot(residual, fill=True, color='blue')
    plt.title(f"Residual Density Plot for {label}")
    plt.xlabel("Residuals")
    plt.ylabel("Density")

plt.tight_layout()
plt.show()

In [ ]:
prepare_submission(pipeline_eng, "augmented_polynomial_submission.csv")

In [39]:
#prepare_submission(pipeline_eng, "reduced_polynomial_submission.csv")

C:\Users\patri\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


## K-Nearest Neighbours

In [31]:
from sklearn.neighbors import KNeighborsRegressor

In [33]:
def validate_knn_regression(X_train, y_train, X_val, y_val, nns=range(30,40)):
    results = {}
    for k in nns:
        pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("regressor", KNeighborsRegressor(k))
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)
        rms = root_mean_squared_error(y_pred, y_val)
        results[k] = rms
    return results
        
# fraction of the training data used for validating KNN neighbour choice
KNN_SUBSET = 1

n_examples_knn = int(X_train.shape[0] * KNN_SUBSET)
examples_knn = np.random.choice(X_train.shape[0], n_examples_knn, replace=False)
X_knn, y_knn = X_train[examples_knn], y_train[examples_knn]
results = validate_knn_regression(X_knn, y_knn, X_val, y_val)
results

{30: 0.8229688612240088,
 31: 0.821571217151439,
 32: 0.8208578818724056,
 33: 0.8204982220856754,
 34: 0.8195408748460857,
 35: 0.8207365815065937,
 36: 0.821180270165275,
 37: 0.820257923304951,
 38: 0.8197082170719893,
 39: 0.8214370915791646}

In [ ]:
#k=10, should've tested for more k numbers
pipeline_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("distance_features", FunctionTransformer(add_features)),
    ("poly", PolynomialFeatures(degree=2)),
    ("regressor", KNeighborsRegressor(10))
])
pipeline_knn.fit(X_train, y_train)
print("Fit!")
prepare_submission(pipeline_knn, "knn_submission.csv")
y_pred = pipeline_knn.predict(X_val)
rms = root_mean_squared_error(y_pred, y_val)
print(rms)

In [ ]:
plot_y_yhat(y_val,y_pred, plot_title = "plot")

In [ ]:
residuals_dict = {
    'x_1': residuals[:, 0],
    'y_1': residuals[:, 1],
    'x_2': residuals[:, 2],
    'y_2': residuals[:, 3],
    'x_3': residuals[:, 4],
    'y_3': residuals[:, 5]
}

# Create a density plot for each residual (x_1, y_1, x_2, y_2, x_3, y_3)
plt.figure(figsize=(12, 8))

for i, (label, residual) in enumerate(residuals_dict.items()):
    plt.subplot(3, 2, i+1)
    sns.kdeplot(residual, fill=True, color='blue')
    plt.title(f"Residual Density Plot for {label}")
    plt.xlabel("Residuals")
    plt.ylabel("Density")

plt.tight_layout()
plt.show()